## READ THE FILES

In [487]:
import pandas as pd

df = pd.read_csv('resources/patient_data.csv', sep=',')
print(df)

                      createdAt                   updatedAt  \
0    2020-01-19 18:40:57.358935  2022-01-21 06:11:55.146941   
1    2021-03-13 08:28:58.732378  2021-06-03 09:12:18.429898   
2    2021-11-26 07:35:12.516895  2021-11-26 07:35:12.516895   
3    2023-04-14 15:33:45.468723  2024-05-20 13:03:30.377979   
4    2020-06-01 14:46:19.545465  2023-05-07 05:15:35.824240   
..                          ...                         ...   
545  2024-05-28 17:38:43.929305  2024-06-17 07:29:17.675220   
546  2023-09-20 20:05:46.498330  2024-12-10 16:36:30.040933   
547  2021-03-05 13:37:39.058296  2023-01-15 07:57:20.531440   
548  2022-08-28 05:58:58.535950  2023-09-08 03:31:00.867427   
549  2020-06-26 23:26:46.269573  2020-06-26 23:26:46.269573   

                                patientID        patientName patientBirthDate  \
0    5d7d8da0-7dbe-4b0a-9c44-8f2fc1f3c51e       Tari Chandra       2006-06-15   
1    d3fe7926-cac1-4a3d-99d0-07f29b8a8f33               Lina       1991-04-28   


In [488]:
#Completeness

completeness = df.isnull().sum().to_frame(name='Missing Count')
completeness['Missing %'] = (completeness['Missing Count'] / len(df)) * 100
completeness

,Missing Count,Missing %
createdAt,0,0.000000
updatedAt,0,0.000000
patientID,0,0.000000
patientName,0,0.000000
patientBirthDate,76,13.818182
patientNIK,47,8.545455
patientPhoneNumber,0,0.000000
patientAddress,0,0.000000
patientEmail,43,7.818182
isAlive,0,0.000000


In [489]:
#Uniqueness

duplicate_rows = df.duplicated().sum()
uniqueness = df.nunique().to_frame(name='Unique Values')
uniqueness['Unique %'] = (uniqueness['Unique Values'] / len(df)) * 100
uniqueness

,Unique Values,Unique %
createdAt,550,100.000000
updatedAt,550,100.000000
patientID,550,100.000000
patientName,321,58.363636
patientBirthDate,352,64.000000
patientNIK,369,67.090909
patientPhoneNumber,416,75.636364
patientAddress,416,75.636364
patientEmail,380,69.090909
isAlive,2,0.363636


In [490]:
#Validation

#Validate date
import re

df_validation = pd.DataFrame(columns=[])

df_validation['createdAt_valid'] = pd.to_datetime(df['createdAt'], errors='coerce')
df_validation['updatedAt_valid'] = pd.to_datetime(df['updatedAt'], errors='coerce')
df_validation['patientBirthDate_valid'] = pd.to_datetime(df['patientBirthDate'], errors='coerce')
invalid_created_at = df_validation['createdAt_valid'].isnull().sum()
invalid_updated_at = df_validation['updatedAt_valid'].isnull().sum()
invalid_patient_birth_date = df_validation['patientBirthDate_valid'].isnull().sum()
print(f"Invalid createdAt: {invalid_created_at}")
print(f"Invalid updatedAt: {invalid_updated_at}")
print(f"Invalid patientBirthDate: {invalid_patient_birth_date} ({invalid_patient_birth_date / len(df) * 100:.2f}%)")
df_invalid_birth_date = df[df_validation['patientBirthDate_valid'].isnull()]['patientBirthDate'].sample(5)

#Validate email
email_pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
df_validation['email_valid'] = df['patientEmail'].apply(lambda x: bool(re.match(email_pattern, str(x))) if pd.notnull(x) else False)
invalid_emails = (~df_validation['email_valid']).sum()
print(f"Invalid emails: {invalid_emails}")
df_invalid_emails = df[df_validation['email_valid'] == False]['patientEmail'].sample(5)

#Validate NIK
df_validation['NIK_valid'] = df['patientNIK'].apply(lambda x: len(str(int(x))) == 16 if not pd.isnull(x) else False)
invalid_nik = (~df_validation['NIK_valid']).sum()
print(f"Invalid NIK: {invalid_nik}")
df_invalid_nik = df[df_validation['NIK_valid'] == False]['patientNIK'].sample(5)

#Validate Phone Number
df_validation['Phone_valid'] = df['patientPhoneNumber'].astype(str).apply(lambda x: x.startswith('62') and 10 <= len(x) <= 14)
invalid_phone = (~df_validation['Phone_valid']).sum()
print(f"Invalid Phone Number: {invalid_phone}")
df_invalid_phone = df[df_validation['Phone_valid'] == False]['patientPhoneNumber'].sample(5)

df_sample = pd.concat([df_invalid_birth_date, df_invalid_emails, df_invalid_nik, df_invalid_phone], axis=1)
df_sample

Invalid createdAt: 0
Invalid updatedAt: 0
Invalid patientBirthDate: 76 (13.82%)
Invalid emails: 43
Invalid NIK: 47
Invalid Phone Number: 167


,patientBirthDate,patientEmail,patientNIK,patientPhoneNumber
134,NaN,NaN,NaN,NaN
444,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN
123,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN
368,NaN,NaN,NaN,NaN
153,NaN,NaN,NaN,NaN
344,NaN,NaN,NaN,NaN


In [491]:
#Consistancy
df_consistency = df.dropna(subset=['updatedAt', 'patientBirthDate'])[['createdAt', 'updatedAt', 'patientBirthDate']]
df_consistency['Date_Consistent'] = df_consistency['updatedAt'] >= df_consistency['createdAt']
inconsistent_dates = (~df_consistency['Date_Consistent']).sum()
print(f"Inconsistent updatedAt: {inconsistent_dates}")

df_consistency['BirthDate_Logic'] = df_consistency['patientBirthDate'] < df_consistency['createdAt']
inconsistent_birth = df_consistency['BirthDate_Logic'].isnull().sum() + (~df_consistency['BirthDate_Logic'].dropna()).sum()
print(f"Inconsistent patientBirthDate: {inconsistent_birth}")

df_consistency = df_consistency[~df_consistency['Date_Consistent'] | ~df_consistency['BirthDate_Logic']]
df_consistency

Inconsistent updatedAt: 0
Inconsistent patientBirthDate: 16


,createdAt,updatedAt,patientBirthDate,Date_Consistent,BirthDate_Logic
19,2020-10-30 12:15:41.704839,2021-12-20 00:36:43.152374,2021-11-22,True,False
20,2021-01-03 04:57:32.746297,2022-12-23 22:04:55.458510,2024-07-30,True,False
109,2024-04-07 08:23:22.902361,2025-01-12 08:45:19.796026,2025-05-11,True,False
143,2021-01-29 11:49:57.037194,2024-12-01 23:57:28.984539,2022-05-20,True,False
168,2021-09-07 20:49:54.729859,2021-09-07 20:49:54.729859,2022-11-15,True,False
187,2023-08-24 03:03:24.081685,2023-08-24 03:03:24.081685,2024-06-28,True,False
226,2022-06-03 18:35:41.041779,2024-02-15 08:12:07.742302,2023-02-25,True,False
254,2020-10-11 11:31:09.593711,2023-04-19 10:02:27.895487,2024-10-11,True,False
269,2021-05-10 09:29:44.339694,2021-06-18 08:40:35.341826,2024-03-20,True,False
318,2020-04-24 18:17:56.522855,2020-04-24 18:17:56.522855,2023-02-25,True,False


In [492]:
created_stats = df['createdAt'].describe()
created_stats['first'] = df['createdAt'].min()
created_stats['last'] = df['createdAt'].max()

df_birthdate = df.dropna(subset=['patientBirthDate']).copy()
df_birthdate['patientBirthDate'] = pd.to_datetime(df_birthdate['patientBirthDate'])
current_date = pd.Timestamp.now()
df_birthdate['patientAge'] = current_date.year - df_birthdate['patientBirthDate'].dt.year
df_age_stats = df_birthdate['patientAge'].describe()

top_emails = df['patientEmail'].value_counts().head(5)
top_patients_names = df['patientName'].value_counts().head(5).index
top_patients_df = df[df['patientName'].isin(top_patients_names)]
top_patients_unique = top_patients_df.drop_duplicates(subset=['patientName'])

print(f"Created Range: \n{created_stats}\n")
print(f"Patient Age: \n{df_birthdate[['patientID', 'patientName', 'patientBirthDate', 'patientAge']]}\n")
print(f"Patient Age Statistics: \n{df_age_stats}\n")
print(f"Top 5 Email Domains: \n{top_emails.to_frame(name='Count')}\n")
print(f"Top 5 Patients: \n{top_patients_unique[['patientID', 'patientName']]}")

Created Range: 
count                            550
unique                           550
top       2020-01-19 18:40:57.358935
freq                               1
first     2020-01-10 10:10:28.359719
last      2024-12-25 13:42:25.267148
Name: createdAt, dtype: object

Patient Age: 
                                patientID        patientName patientBirthDate  \
0    5d7d8da0-7dbe-4b0a-9c44-8f2fc1f3c51e       Tari Chandra       2006-06-15   
1    d3fe7926-cac1-4a3d-99d0-07f29b8a8f33               Lina       1991-04-28   
4    b8b8d41f-f048-44b8-9940-99a07229213e     Tono Oktaviani       2002-08-29   
5    2a5c8c18-83ba-4ebd-87cc-7e4cef84a4cc          Aldi Sari       1953-10-07   
6    e1b92377-fc64-4d7b-9e11-f6c61622731f       Deni Santoso       2009-01-23   
..                                    ...                ...              ...   
544  43a848a1-fd47-4fc9-9d33-7bdef992157b    Fitri Oktaviani       1963-04-22   
546  6e5051b1-9922-4fa5-a6ef-4414fec8a400     Reza Kurniawan       1

## Analyse Duplicate

In [ ]:
from rapidfuzz import fuzz
from itertools import combinations

def fuzzy_email_match(email1, email2):
    try:
        user1, domain1 = email1.lower().split('@')
        user2, domain2 = email2.lower().split('@')
    except:
        return 0  # skip invalid email

    user_score = fuzz.token_sort_ratio(user1, user2)
    domain_score = fuzz.ratio(domain1, domain2)

    total_score = (0.7 * user_score + 0.3 * domain_score)
    return total_score

cols = ['patientID', 'patientName', 'patientNIK', 'patientBirthDate', 'patientAddress', 'patientEmail']
df_check = df[cols].copy()
matches = []

for col in ['patientNIK', 'patientName', 'patientBirthDate', 'patientAddress']:
    df_check[col] = df_check[col].astype(str).str.strip()

for (idx1, row1), (idx2, row2) in combinations(df_check.iterrows(), 2):
    name_score = fuzz.token_sort_ratio(row1['patientName'], row2['patientName'])
    address_score = fuzz.partial_ratio(row1['patientAddress'], row2['patientAddress'])
    email_score = fuzzy_email_match(row1['patientEmail'], row2['patientEmail'])
    
    nik_score = None
    if row1['patientNIK'] != 'nan' and row2['patientNIK'] != 'nan':
        nik_score = fuzz.ratio(row1['patientNIK'], row2['patientNIK'])

    birth_match = row1['patientBirthDate'] == row2['patientBirthDate']

    if name_score >= 80 and birth_match:
        matches.append({
            "patientID_1": row1['patientID'],
            "patientID_2": row2['patientID'],
            "Name Score": name_score,
            "NIK Score": nik_score,
            "Address Score": address_score,
            "Email Score": email_score,
            "Birth Match": birth_match
        })
        
match_df = pd.DataFrame(matches)
match_df

,patientID_1,patientID_2,Name Score,NIK Score,Address Score,Email Score,Birth Match
0,74799bc6-1d4b-4dd7-a164-09b2a57b5d6d,fe373645-ace4-4d93-9108-377553e67229,100.000000,100.000000,100.000000,100.0,True
1,74799bc6-1d4b-4dd7-a164-09b2a57b5d6d,0a7fa967-f452-483c-869c-1b0978026e06,100.000000,100.000000,100.000000,100.0,True
2,74799bc6-1d4b-4dd7-a164-09b2a57b5d6d,547bc1f2-e0b0-49cd-b85b-68553b13849c,100.000000,100.000000,100.000000,100.0,True
3,2a5c8c18-83ba-4ebd-87cc-7e4cef84a4cc,dec78847-5bc3-4f84-8958-0c1b53c28ad9,100.000000,100.000000,100.000000,100.0,True
4,8f58e67a-8746-45c9-a391-fc3c76470c86,3c8178d3-d922-4ef7-955a-6aa073f5e6a2,100.000000,44.444444,60.162602,40.0,True
...,...,...,...,...,...,...,...
128,947b2f10-7112-4e79-af45-48f6a1e2be52,2389d8a0-cafb-4580-abd9-386e802ed05e,100.000000,100.000000,100.000000,100.0,True
129,947b2f10-7112-4e79-af45-48f6a1e2be52,e7234819-336c-4701-8d08-e1a8f420bae8,82.758621,100.000000,100.000000,100.0,True
130,2389d8a0-cafb-4580-abd9-386e802ed05e,e7234819-336c-4701-8d08-e1a8f420bae8,82.758621,100.000000,100.000000,100.0,True
131,a175fb9f-9776-4af2-bb46-871e8d957103,db86495f-2bbf-4b03-a664-cf6b092a845c,82.758621,100.000000,100.000000,100.0,True


In [494]:
merged_rows = []
merged_ids = set()

for _, match in match_df.iterrows():
    id1, id2 = match['patientID_1'], match['patientID_2']
    if id1 in merged_ids or id2 in merged_ids:
        continue

    row1 = df[df['patientID'] == id1].iloc[0]
    row2 = df[df['patientID'] == id2].iloc[0]

    merged = {}
    for col in df.columns:
        val1 = row1[col]
        val2 = row2[col]
        merged[col] = val1 if pd.notnull(val1) and val1 != '' else val2

    merged_rows.append(merged)
    merged_ids.update([id1, id2])

cleaned_df = pd.DataFrame(merged_rows)
df_final = pd.concat([df[~df['patientID'].isin(merged_ids)], cleaned_df], ignore_index=True)
df_final

,createdAt,updatedAt,patientID,patientName,patientBirthDate,patientNIK,patientPhoneNumber,patientAddress,patientEmail,isAlive
0,2020-01-19 18:40:57.358935,2022-01-21 06:11:55.146941,5d7d8da0-7dbe-4b0a-9c44-8f2fc1f3c51e,Tari Chandra,2006-06-15,1.292365e+15,850926551,"Perum Gang S. Parman No. 910, RT 13/RW 07, Vil...",NaN,True
1,2021-03-13 08:28:58.732378,2021-06-03 09:12:18.429898,d3fe7926-cac1-4a3d-99d0-07f29b8a8f33,Lina,1991-04-28,1.282961e+15,841959086,"Gang Gg. Tebet Barat Dalam No. 697, RT 20/RW 0...",lina175@yahoo.co.id,False
2,2021-11-26 07:35:12.516895,2021-11-26 07:35:12.516895,342ee413-dc71-4914-a854-0c0af156d324,Budi Susanti,NaN,1.197592e+15,830288623,"Komplek Jalan Dipenogoro No. 562, RT 16/RW 07,...",budisusanti270@yahoo.co.id,True
3,2020-06-01 14:46:19.545465,2023-05-07 05:15:35.824240,b8b8d41f-f048-44b8-9940-99a07229213e,Tono Oktaviani,2002-08-29,NaN,62860684795,"Gang Gang Pacuan Kuda No. 488, RT 18/RW 06, Vi...",tonooktaviani95@gmail.com,True
4,2022-02-16 03:45:44.195849,2022-02-16 03:45:44.195849,e1b92377-fc64-4d7b-9e11-f6c61622731f,Deni Santoso,2009-01-23,1.349442e+15,62804873559,"Gg. Gg. Pasir Koja No. 643, RT 15/RW 04, Ville...",denisantoso68@yahoo.com,True
...,...,...,...,...,...,...,...,...,...,...
480,2020-06-17 04:50:13.176697,2022-03-08 05:43:39.790163,224d4070-7e2d-43dc-8593-5f1ae967029d,Bayu,1950-02-23,5.362176e+15,62882767315,"Gang Jalan Cikapayang No. 726, RT 11/RW 03, Vi...",bayu689@yahoo.com,True
481,2024-06-20 05:14:28.684375,2024-10-01 15:08:05.782154,977e3a10-0478-4d2a-b694-39558502df69,Rani Fitriani,2007-04-01,3.508447e+15,62887381253,"Gg. Gg. Kapten Muslihat No. 923, RT 09/RW 04, ...",ranifitriani846@gmail.com,True
482,2024-10-01 00:33:07.145224,2024-10-01 00:33:07.145224,62c8874a-19a4-4178-943b-dc07a92de809,Vina Fitriani Ayu,2009-06-02,6.555742e+15,62851247377,"Jalan Jalan Bangka Raya No. 202, RT 08/RW 02, ...",vinafitriani427@gmail.co.id,True
483,2020-02-02 20:23:26.026502,2021-12-04 09:57:22.963942,493c7763-b9c0-48e4-b66c-e6aec63b13b2,Bayu,1950-02-23,5.362176e+15,62882767315,"Gang Jalan Cikapayang No. 726, RT 11/RW 03, Vi...",bayu689@yahoo.com,True


In [495]:
completeness = df_final.isnull().sum().to_frame(name='Missing Count')
completeness['Missing %'] = (completeness['Missing Count'] / len(df)) * 100
completeness

,Missing Count,Missing %
createdAt,0,0.000000
updatedAt,0,0.000000
patientID,0,0.000000
patientName,0,0.000000
patientBirthDate,64,11.636364
patientNIK,45,8.181818
patientPhoneNumber,0,0.000000
patientAddress,0,0.000000
patientEmail,39,7.090909
isAlive,0,0.000000


In [496]:
duplicate_rows = df_final.duplicated().sum()
uniqueness = df_final.nunique().to_frame(name='Unique Values')
uniqueness['Unique %'] = (uniqueness['Unique Values'] / len(df_final)) * 100
uniqueness['Unique Values old data'] = df.nunique()
uniqueness['Unique % old data'] = (uniqueness['Unique Values old data'] / len(df)) * 100
uniqueness

,Unique Values,Unique %,Unique Values old data,Unique % old data
createdAt,485,100.000000,550,100.000000
updatedAt,485,100.000000,550,100.000000
patientID,485,100.000000,550,100.000000
patientName,309,63.711340,321,58.363636
patientBirthDate,352,72.577320,352,64.000000
patientNIK,364,75.051546,369,67.090909
patientPhoneNumber,409,84.329897,416,75.636364
patientAddress,409,84.329897,416,75.636364
patientEmail,374,77.113402,380,69.090909
isAlive,2,0.412371,2,0.363636


In [497]:
import numpy as np

def clean_phone(phone):
    if pd.isnull(phone):
        return None
    phone = str(phone)
    phone = re.sub(r"[^\d]", "", phone)
    if phone.startswith("0"):
        phone = "62" + phone[1:]
    elif phone.startswith("8"):
        phone = "62" + phone
    elif phone.startswith("+62"):
        phone = phone.replace("+", "")
    return phone

def clean_nik(nik):
    if pd.isnull(nik):
        return nik
    nik = re.sub(r"[^\d]", "", str(nik))
    return nik.zfill(16) if len(nik) <= 16 else nik[:16]

def quality_score(row):
    score = 100

    nik = row['patientNIK']
    if pd.isnull(nik) or len(str(nik)) != 16:
        score -= 25

    phone = row['patientPhoneNumber']
    if pd.isnull(phone) or not str(phone).startswith('62') or len(str(phone)) < 10:
        score -= 20

    if pd.isnull(row['patientName']):
        score -= 20
    if pd.isnull(row['patientBirthDate']):
        score -= 15
    if pd.isnull(row['patientAddress']) or len(str(row['patientAddress'])) < 10:
        score -= 10

    # Email optional (small penalty)
    if pd.isnull(row['patientEmail']):
        score -= 5

    return max(score, 0)

In [501]:
df_final['patientPhoneNumber'] = df_final['patientPhoneNumber'].apply(clean_phone)
df_final['patientNIK'] = df_final['patientNIK'].apply(clean_nik)

# Score
df_final['qualityScore'] = df_final.apply(quality_score, axis=1)
df_final

,createdAt,updatedAt,patientID,patientName,patientBirthDate,patientNIK,patientPhoneNumber,patientAddress,patientEmail,isAlive,qualityScore
0,2020-01-19 18:40:57.358935,2022-01-21 06:11:55.146941,5d7d8da0-7dbe-4b0a-9c44-8f2fc1f3c51e,Tari Chandra,2006-06-15,1292364801956902,62850926551,"Perum Gang S. Parman No. 910, RT 13/RW 07, Vil...",NaN,True,95
1,2021-03-13 08:28:58.732378,2021-06-03 09:12:18.429898,d3fe7926-cac1-4a3d-99d0-07f29b8a8f33,Lina,1991-04-28,1282961406191039,62841959086,"Gang Gg. Tebet Barat Dalam No. 697, RT 20/RW 0...",lina175@yahoo.co.id,False,100
2,2021-11-26 07:35:12.516895,2021-11-26 07:35:12.516895,342ee413-dc71-4914-a854-0c0af156d324,Budi Susanti,NaN,1197592402605899,62830288623,"Komplek Jalan Dipenogoro No. 562, RT 16/RW 07,...",budisusanti270@yahoo.co.id,True,85
3,2020-06-01 14:46:19.545465,2023-05-07 05:15:35.824240,b8b8d41f-f048-44b8-9940-99a07229213e,Tono Oktaviani,2002-08-29,NaN,62860684795,"Gang Gang Pacuan Kuda No. 488, RT 18/RW 06, Vi...",tonooktaviani95@gmail.com,True,75
4,2022-02-16 03:45:44.195849,2022-02-16 03:45:44.195849,e1b92377-fc64-4d7b-9e11-f6c61622731f,Deni Santoso,2009-01-23,1349441611558614,62804873559,"Gg. Gg. Pasir Koja No. 643, RT 15/RW 04, Ville...",denisantoso68@yahoo.com,True,100
...,...,...,...,...,...,...,...,...,...,...,...
480,2020-06-17 04:50:13.176697,2022-03-08 05:43:39.790163,224d4070-7e2d-43dc-8593-5f1ae967029d,Bayu,1950-02-23,5362176411465438,62882767315,"Gang Jalan Cikapayang No. 726, RT 11/RW 03, Vi...",bayu689@yahoo.com,True,100
481,2024-06-20 05:14:28.684375,2024-10-01 15:08:05.782154,977e3a10-0478-4d2a-b694-39558502df69,Rani Fitriani,2007-04-01,3508446712844124,62887381253,"Gg. Gg. Kapten Muslihat No. 923, RT 09/RW 04, ...",ranifitriani846@gmail.com,True,100
482,2024-10-01 00:33:07.145224,2024-10-01 00:33:07.145224,62c8874a-19a4-4178-943b-dc07a92de809,Vina Fitriani Ayu,2009-06-02,6555741812839100,62851247377,"Jalan Jalan Bangka Raya No. 202, RT 08/RW 02, ...",vinafitriani427@gmail.co.id,True,100
483,2020-02-02 20:23:26.026502,2021-12-04 09:57:22.963942,493c7763-b9c0-48e4-b66c-e6aec63b13b2,Bayu,1950-02-23,5362176411465438,62882767315,"Gang Jalan Cikapayang No. 726, RT 11/RW 03, Vi...",bayu689@yahoo.com,True,100


In [505]:
def analyze_name_quality(name):
    if pd.isnull(name) or name.strip() == "":
        return "Missing"
    elif len(name.split()) == 1:
        return "OneWord"
    elif re.search(r'\d', name) or re.search(r'[^\w\s]', name):
        return "ContainsNumberOrSymbol"
    elif name.lower() in ['test', 'xxx', 'abc', 'dummy', 'na', 'unknown']:
        return "InvalidCommon"
    else:
        return "Valid"

df_final['nameQuality'] = df_final['patientName'].apply(analyze_name_quality)

df_final
df_final.to_csv('resources/df_final.csv', index=False)

In [504]:
missing_counts = df_final.isna().sum().reset_index()
missing_counts.columns = ['ColumnName', 'MissingCount']
missing_counts = missing_counts[missing_counts['MissingCount'] > 0]
missing_counts
missing_counts.to_csv('resources/missing_counts.csv', index=False)